In [1]:
%matplotlib inline  

import matplotlib
matplotlib.use("Qt4Agg")
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import sqlite3
from time import time
import audioanalytics
reload(audioanalytics)
from audioanalytics import *
import pca
reload(pca)
from pca import Pca
import tables

from IPython.html.widgets import FloatProgress
from IPython.display import display


/usr/lib/python2.7/dist-packages/matplotlib/__init__.py:1314: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/usr/local/share/.virtualenvs/sc-recom/local/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [ ]:
#database
conn = sqlite3.connect("/home/kayibal/sc-recom/code/msd_data/analysis_data/sampled_likes_tracks.db")
c = conn.cursor()
c.execute("SELECT msd_id, audio_path FROM tracks WHERE audio_path IS NOT NULL ORDER BY msd_id")
tracks = c.fetchall()
conn.close()

In [ ]:
l = len(tracks)
it = 1.
durations = []

f = FloatProgress(min=0, max=100)
display(f)

small = []
error = []
for msd_id, path in tracks:
    print "%s:" % path
    start = time()
    abs_path = os.path.join("/home/kayibal/sc-recom/code/msd_data/analysis_data",path)
    save_path = os.path.dirname(abs_path)
    
    if not os.path.exists(os.path.join(save_path,msd_id+"_fluc.npy")):
        try:
            fp = FluctuationPattern(abs_path,chunk=16,Hz=11025)
            fm = fp.get_feature_matrix()
            np.save(os.path.join(save_path,msd_id+"_fluc"),fm)
        except SmallFileError,e:
            small.append(msd_id)
            continue
        except KeyboardInterrupt:
            raise
        except Exception,e:
            error.append((msd_id,path,e))
            continue
    if not os.path.exists(os.path.join(save_path,msd_id+"_mfcc.npy")):
        try:
            mfcc = MFCC(abs_path,mel_bands=24)
            np.save(os.path.join(save_path,msd_id+"_mfcc"),mfcc.mfcc)
        except SmallFileError,e:
            small.append(msd_id)
            continue
        except KeyboardInterrupt:
            raise
        except Exception,e:
            error.append((msd_id,path,e))
            continue
        durations.append(time()- start)
    if not os.path.exists(os.path.join(save_path,msd_id+"_mfcc_delta.npy")):
        try:
            mfcc = MFCC(abs_path,mel_bands=24)
            np.save(os.path.join(save_path,msd_id+"_mfcc"),mfcc.mfcc)
        except SmallFileError,e:
            small.append(msd_id)
            continue
        except KeyboardInterrupt:
            raise
        except Exception,e:
            error.append((msd_id,path,e))
            continue
        durations.append(time()- start)
    rem_time = np.mean(durations) * (l - it)
    f.value = (it/l)*100
    print "progress %f remaining time %s" % ((it/l)*100, str(round(rem_time/60.,3)))
    it = it + 1.

In [ ]:
#database
conn = sqlite3.connect("/home/kayibal/sc-recom/code/msd_data/analysis_data/sampled_likes_tracks.db")
c = conn.cursor()
#remove failed files from database
query = "DELETE FROM tracks WHERE msd_id IN (%s)" % "'TRLHQBM128F92D31BD', 'TRRJMYM128F4259501', 'TRRVWMI128F92E7D38','TRTOACI128F42819C7', 'TRBHQRU128F93129F9', 'TRIYPSF12903CDFAB8', 'TRNNKZA128F4226422'"
c.execute(query)
conn.commit()
conn.close()

In [ ]:
#collect all data
try:
    f = tables.open_file("/home/kayibal/sc-recom/code/msd_data/analysis_data/analysis2.h5", mode="w", title="Audio Analysis File")
    f.create_group('/', 'analysis', "Audio Analysis Data Matrices")

    filters = tables.Filters(complevel=5, complib='blosc')
    dt = tables.Atom.from_dtype(np.dtype('float64'))

    flucs_data = f.createCArray(f.root.analysis, 'flucs',
                                          dt,
                                          shape=(1440,len(tracks)),
                                          filters=filters)

    mfcc_data = f.createEArray(f.root.analysis, 'mfccs',
                              dt,
                              shape=(14, 0),
                              filters=filters,
                              expectedrows=15e+6)

    mfcc_dyn_data = f.createEArray(f.root.analysis, 'mfcc_dyns',
                                  dt,
                                  shape=(131, 0),
                                  filters=filters,
                                  expectedrows=15e+5)

    mfcc_delta_data = f.createEArray(f.root.analysis, 'mfcc_deltas',
                                      dt,
                                      shape=(14, 0),
                                      filters=filters,
                                      expectedrows=15e+6)

    mfcc_delta_dyn_data = f.createEArray(f.root.analysis, 'mfcc_delta_dyns',
                                          dt,
                                          shape=(131, 0),
                                          filters=filters,
                                          expectedrows=15e+5)

    mfcc_median_data = f.createCArray(f.root.analysis, 'mfcc_medians',
                                      dt,
                                      shape=(130,len(tracks)),
                                      filters=filters)
    mfcc_median_delta_data = f.createCArray(f.root.analysis, 'mfcc_delta_medians',
                                            dt,
                                            shape=(130,len(tracks)),
                                            filters=filters)

    f = FloatProgress(min=0, max=100)
    display(f)

    idx = 0
    for msd_id, path in tracks:
        #print "%s:" % path
        start = time()
        abs_path = os.path.join("/home/kayibal/sc-recom/code/msd_data/analysis_data",path)
        save_path = os.path.dirname(abs_path)
        fp = np.load(os.path.join(save_path,msd_id+"_fluc.npy"))
        mfcc = np.load(os.path.join(save_path,msd_id+"_mfcc.npy"))
        mfcc_delta = np.load(os.path.join(save_path,msd_id+"_mfcc_delta.npy"))
        flucs_data[:,idx] = fp.flatten()

        #add index information to mfcc data
        mfcc_idx = np.zeros((14,mfcc.shape[1]))
        mfcc_idx[0,:] = idx
        mfcc_idx[1:,:] = mfcc
        mfcc_data.append(mfcc_idx)

        mfcc_delta_idx = np.zeros((14,mfcc.shape[1]))
        mfcc_delta_idx[0,:] = idx
        mfcc_idx[1:,:] = mfcc_delta
        mfcc_delta_data.append(mfcc_delta_idx)

        #reshape mfcc
        length = floor(mfcc.shape[1]/10.)*10
        temp = np.zeros((131,length/10))
        temp[0,:] = idx
        temp[1:,:] = mfcc[:,:length].reshape((130,-1),order="F")
        mfcc_median_data[:,idx] = np.median(temp[1:,:], axis=1)
        mfcc_dyn_data.append(temp)

        length = floor(mfcc_delta.shape[1]/10.)*10
        temp = np.zeros((131,length/10))
        temp[0,:] = idx
        temp[1:,:] = mfcc_delta[:,:length].reshape((130,-1),order="F")
        mfcc_median_delta_data[:,idx] = np.median(temp[1:,:], axis=1)
        mfcc_delta_dyn_data.append(temp)

        idx += 1
        f.value = float(idx)/len(tracks) * 100
except Exception:
    f.close()
    raise

In [ ]:
fluc_pca = Pca(flucs_data.transpose())

In [ ]:
comp = fluc_pca.project_data(flucs_data,0.70)

In [ ]:
np.unique(np.where(np.isnan(mfccs_mean_data))[1])

In [ ]:
m = MFCC(os.path.join("/home/kayibal/sc-recom/code/msd_data/analysis_data",tracks[3988][1]))
mfcc = m.mfcc
length = floor(mfcc.shape[1]/10.)*10
temp = mfcc[:,:length].reshape((130,-1),order="F")
mfccs_mean_data[:,3988] = np.mean(temp, axis=1)
mfccs_median_data[:,3988] = np.median(temp, axis=1)

In [ ]:
m = MFCC(os.path.join("/home/kayibal/sc-recom/code/msd_data/analysis_data",tracks[6312][1]))
mfcc = m.mfcc
length = floor(mfcc.shape[1]/10.)*10
temp = mfcc[:,:length].reshape((130,-1),order="F")
mfccs_mean_data[:,6312] = np.mean(temp, axis=1)
mfccs_median_data[:,6312] = np.median(temp, axis=1)

In [ ]:
mfcc_mean_pca = Pca(mfccs_mean_data.transpose())
mfcc_median_pca = Pca(mfccs_median_data.transpose())

In [ ]:
mfcc_mean_comp = mfcc_mean_pca.project_data(mfccs_mean_data,0.9)
mfcc_median_comp = mfcc_median_pca.project_data(mfccs_median_data,0.85)

In [ ]:
mfcc_median_comp.shape

In [ ]:
A_features = np.vstack((comp,mfcc_mean_comp)) 
B_features = np.vstack((comp,mfcc_median_comp))

In [ ]:
C_features = np.vstack((comp,mfcc_mean_comp, mfcc_median_comp)) 

In [ ]:
f = tables.open_file("/home/kayibal/sc-recom/code/msd_data/analysis_data/analysis.h5", mode="w", title="Audio Analysis File")
atom = tables.Atom.from_dtype(flucs_data.dtype)
f.create_group('/', 'analysis', "Audio Analysis Data Matrices")
fd = f.create_carray(f.root.analysis, "fluctuations", atom, flucs_data.shape)
m1 = f.create_carray(f.root.analysis, "mfcc_means", atom, mfccs_mean_data.shape)
m2 = f.create_carray(f.root.analysis, "mfcc_medians", atom, mfccs_median_data.shape)
A = f.create_carray(f.root.analysis, "A_features", atom, A_features.shape)
B = f.create_carray(f.root.analysis, "B_features", atom, B_features.shape)
C = f.create_carray(f.root.analysis, "C_features", atom, C_features.shape)
fd[:] = flucs_data
m1[:] = mfccs_mean_data
m2[:] = mfccs_median_data
A[:] = A_features
B[:] = B_features
C[:] = C_features
f.close()

In [ ]:
msd_ids = np.array([t[0] for t in tracks])

f = tables.open_file("/home/kayibal/sc-recom/code/msd_data/analysis_data/analysis.h5", mode="a")
f.create_group('/', 'metadata', "Audio Analysis Data Matrices")
atom = tables.Atom.from_dtype(msd_ids.dtype)
ids = f.create_carray(f.root.analysis, "msd_ids", atom, msd_ids.shape)
ids[:] = msd_ids
f.close()

In [ ]:
m = np.arange(100).reshape((10,10))
m2 = np.arange(100)

In [ ]:
m[3,3]

In [ ]:
np.ravel_multi_index((3,3),(10,10))

In [ ]:
f.close()

In [ ]:
from sklearn.decomposition import IncrementalPCA as IPCA
#pca over mfcc dynamics
ipca = IPCA(n_components = 20)
f = tables.open_file("/home/kayibal/sc-recom/code/msd_data/analysis_data/analysis2.h5", mode="a")


length = f.root.analysis.mfcc_delta_dyns.shape[1]
dt = tables.Atom.from_dtype(np.dtype('float64'))
filters = tables.Filters(complevel=5, complib='blosc')
source = f.root.analysis.mfcc_dyns
target = f.root.analysis.dyns_comp

#source = f.root.analysis.mfcc_dyns
#target = f.root.analysis.mfcc_dyn_comp


chunk_size = 6790
chunk = np.asarray([0,chunk_size])
times = int(floor(float(length) / chunk_size))

prog = FloatProgress(min=0, max=100)
display(prog)

rng = chunk

for i,r in enumerate(source.iterrows(step=chunk_size)):
    ipca.fit(np.asarray(source.read(start=rng[0], stop= rng[1]))[1:,:].transpose())
    prog.value = (float(i)/times)*100.
    rng = rng+chunk_size
    
prog = FloatProgress(min=0, max=100)
display(prog)

rng = chunk
for i,r in enumerate(source.iterrows(step=chunk_size)):
    print rng, target.shape[1]
    target.append(ipca.transform(np.asarray(source.read(start=rng[0], stop= rng[1]))[1:,:].transpose()).transpose())
    prog.value = (float(i)/times)*100.
    rng = rng+chunk_size

f.close()


<h2>Feature Selection</h2>
<p>After compressing the data we can select the most common features for each Song</p>

In [ ]:
def get_song(data, indizes, start_idx = 0, end_idx = 400):
    #first get the data from the array
    mult = 2
    while len(np.unique(indizes[start_idx:end_idx])) <= 1 and end_idx < len(indizes):
        end_idx += end_idx * mult
        mult = mult**2
    if end_idx >= len(indizes) and end_idx - start_idx < 1e4:
        end_idx = len(indizes)-1
    song_idx = np.unique(indizes[start_idx:end_idx])[0]
    song_frame = np.where(indizes[start_idx:end_idx]==song_idx)[0]
    return song_frame[-1], data[:,song_frame[0]:song_frame[-1]]

In [ ]:
f = tables.open_file("/home/kayibal/sc-recom/code/msd_data/analysis_data/analysis2.h5", mode="r")
indizes = f.root.analysis.mfcc_dyns[0,:]
data = f.root.analysis.dyns_comp
end_idx, song = get_song(data,indizes)

In [ ]:
end_idx

In [ ]:
f.close()

In [ ]:
test = []
for i in range(30):
    test.append(range(20))


In [ ]:
test = np.asarray(test)
np.sum(test, axis=1)

In [5]:
#get datarange
maxs = []
mins = []
f = tables.open_file("/home/kayibal/sc-recom/code/msd_data/analysis_data/analysis2.h5", mode="r")
source = f.root.analysis.mfccs
for i,r in enumerate(source.iterrows(10000)):
    if source.read(start=i*10000, stop=-1+(i+1)*10000).shape[1] > 0:
        maxs.append(np.max(np.asarray(source.read(start=i*10000, stop=-1+(i+1)*10000)), axis=1))
        mins.append(np.min(np.asarray(source.read(start=i*10000, stop=-1+(i+1)*10000)), axis=1))
f.close()
source = None
dataRange = np.max(np.asarray(maxs),axis=0) - np.min(np.asarray(mins),axis=0)

In [7]:
dataRange

array([  8.00200000e+03,   5.84141838e+01,   2.08015485e+01,
         1.65209375e+01,   1.94503039e+01,   1.35371962e+01,
         9.29015120e+00,   1.23930666e+01,   1.13649391e+01,
         9.03732514e+00,   7.11169440e+00,   7.03016642e+00,
         7.68339751e+00,   5.98430015e+00])

In [3]:
from igmn import IGMN

In [ ]:
m = IGMN(dataRange=dataRange, tau=0.15, delta=0.1, spmin=2, vmin=10)
f = tables.open_file("/home/kayibal/sc-recom/code/msd_data/analysis_data/analysis2.h5", mode="r")
source = f.root.analysis.mfccs

prog = FloatProgress(min=0, max=100)
display(prog)

for i,r in enumerate(source.iterrows(step=1000)):
    m.train(np.asarray(source.read(start=i*1000, stop=-1+(i+1)*1000)).transpose())
    prog.value = float(i*1000)*100/source.shape[1]

In [ ]:
from sklearn import mixture
m = mixture.GMM(n_components=64, covariance_type="full")
f = tables.open_file("/home/kayibal/sc-recom/code/msd_data/analysis_data/analysis2.h5", mode="r")
source = f.root.analysis.mfccs

prog = FloatProgress(min=0, max=100)
display(prog)

for i,r in enumerate(source.iterrows(step=1000)):
    m.fit(np.asarray(source.read(start=i*1000, stop=-1+(i+1)*1000)).transpose())
    prog.value = float(i*1000)*100/source.shape[1]

In [17]:
f.close()

In [ ]:
np.sum(m.get_song_rep(song.transpose()), axis=0)

In [ ]:
f = tables.open_file("/home/kayibal/sc-recom/code/msd_data/analysis_data/analysis2.h5", mode="a")
source = f.root.analysis.dyns_comp
target = f.root.analysis.gaussian_rep
indizes = f.root.analysis.mfcc_dyns[0,:]

song_idx = np.zeros((2,8003))
for i in range(8003):
    idx = np.where(indizes==i)[0]
    song_idx[0,i] = idx[0]
    song_idx[1,i] = idx[-1]

In [ ]:
f.close()

In [ ]:
song_idx[1,:]

In [ ]:
f = tables.open_file("/home/kayibal/sc-recom/code/msd_data/analysis_data/analysis2.h5", mode="a")
source = f.root.analysis.dyns_comp
target = f.root.analysis.gaussian_rep

for i in range(8003):
    song = source[:,song_idx[i,0]:song_idx[i,1]]
    target[:,i] = np.sum(m.get_song_rep(song.transpose()), axis=0)
    print i
f.close()

In [ ]:
f = tables.open_file("/home/kayibal/sc-recom/code/msd_data/analysis_data/analysis2.h5", mode="a")
np.save("/home/kayibal/sc-recom/code/msd_data/analysis_data/gaussian_rep", np.asarray(f.root.analysis.gaussian_rep))

In [ ]:
f = tables.open_file("/home/kayibal/sc-recom/code/msd_data/analysis_data/analysis2.h5", mode="a")

ipca1 = IPCA(n_components = 40)
ipca2 = IPCA(n_components = 40)
medians_comp[:] = ipca1.fit_transform(np.asarray(f.root.analysis.mfcc_medians).transpose()).transpose()
delta_comp[:] = ipca2.fit_transform(np.asarray(f.root.analysis.mfcc_delta_medians).transpose()).transpose()
f.close()

In [ ]:
%debug

In [ ]:
f = tables.open_file("/home/kayibal/sc-recom/code/msd_data/analysis_data/analysis2.h5", mode="a")
f.root.analysis

In [ ]:
f.close()

In [ ]:
f.close()